# Online Environment for Synergistic Simulations

In this notebook, a general, modifiable environment will be displayed below for experiments with LLMs working together to solve a coding assignment. The coding assignment can be changed, as well as the number of agents. 

This notebook utilizes the ReflectiveLLM class from the multiagents-world2.ipynb class.

## Import Statements

In [ ]:
from agents import ReflectiveLLM, resp_to_json

# Read in the prompt from the prompt.txt file

In [ ]:
with open("prompt.txt", "r") as prompt_file:
    prompt = prompt_file.read()

In [ ]:
rllm = ReflectiveLLM(
    'You are an expert problem solver tasked with breaking up a coding question prompt into smaller parts for other agents to tackle.'
    + 'The parts should be able to be completed in around 20-30 lines of code'
    + 'Do not add any extra information besides for the prompt. If a step requires refactoring previous code (i.e. doing something in a loop), state that explicitly',
    temperature=0.1
)

In [ ]:
response = rllm('The prompt: ' + prompt, 
                True,
                response_format={'list of steps': 'a list of strings, with each element being a step'})

In [ ]:
response

In [ ]:
steps = resp_to_json(response)['steps']
steps

The event loop should be as follows:

The agents are given a prompt to work on - this will include the coding language to be written in. They will have a file path to write code to and access. 

How will the parts be split up - by the agents themselves or by a moderator LLM? important decision there

The order -> the agents will go at the problem sequentially, and be able to converse with each other/add comments about their code or comment on others' code.

Evaluation -> How to evaluate the code that is written? Compare to just one LLM doing the prompt, run tests on it, we'll see

Can ask agents to do one themselves, and one with another agent, and ask them to reflect on both experiences.

Should more information be added about the agents? Experience, etc. -> adding trolls into the mix? maybe not now


In [ ]:
# Set up agents
agents = []
agents.append(ReflectiveLLM(
    'You are tasked with working together with other LLM agents to solve a coding problem. You will not repeat code under any circumstances.' +
    'You will be given tasks to complete in a certain programming language, and will provide the code and nothing but the code.'
    'The overall prompt is as follows: ' + prompt,
    temperature=0.2
))
agents.append(ReflectiveLLM(
    'You are tasked with working together with other LLM agents to solve a coding problem. You will not repeat code under any circumstances.' +
    'You will be given tasks to complete in a certain programming language, and will provide the code and nothing but the code.'
    'The overall prompt is as follows: ' + prompt,
    temperature=0.15
))

In [ ]:
step_count = 0
code = []
code_block = ""
agent_num = 0

while step_count < len(steps): #placeholder for number of times to go through the loop
    code.append(resp_to_json(agents[agent_num]('Here is the previous code for reference: ' + code_block + 'You will ONLY take and ONLY COMPLETE this step. DO NOT DO ANY MORE THAN THIS STEP: ' + steps[step_count],
                False,
                response_format={'Code': 'Your Python code for ONLY this ONE step' + steps[step_count] + ' for this problem. DO NOT INCLUDE MORE CODE THAN FOR JUST THE STEP',
                                 'Chatroom Message': 'A string message for "chatroom" to be visible by other agents. Keep these short and helpful, these messages may be viewed by other agents.'})))
    code_block += code[step_count]['Code']
    print(code[step_count]['Chatroom Message'])
    print(code[step_count]['Code'])
    step_count += 1
    agent_num += 1
    if agent_num == len(agents):
        agent_num = 0

In [ ]:
code

In [ ]:
with open("code.txt", "w") as output_file:
    output_file.write(code_block)